In [1]:
import numpy as np
import pandas as pd
import re

from IPython.display import display
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize  
from collections import Counter
import string

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter

import os

from tqdm import tqdm
import ast
from math import log


from gensim.models import Word2Vec
import random

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
twitter = Twitter()

C:\Users\Samsung\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [3]:
## dictionary from https://github.com/mrlee23/KoreanSentimentAnalyzer

dic_extype = pd.read_csv(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\sentiment analysis\documents\KoreanSentimentAnalyzer-master\KoreanSentimentAnalyzer-master\dic\expressive-type.csv")
dic_inten = pd.read_csv(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\sentiment analysis\documents\KoreanSentimentAnalyzer-master\KoreanSentimentAnalyzer-master\dic\intensity.csv")
dic_polar = pd.read_csv(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\sentiment analysis\documents\KoreanSentimentAnalyzer-master\KoreanSentimentAnalyzer-master\dic\polarity.csv")


In [4]:
dic_extype.head()

,ngram,freq,dir-action,dir-explicit,dir-speech,indirect,writing-device,max.value,max.prop
0,가*/JKS,1,0.0,0.000000,1.000000,0.000000,0.000000,dir-speech,1.000000
1,가*/VV,3,0.0,0.333333,0.333333,0.333333,0.000000,dir-explicit,0.333333
2,가/JKC,17,0.0,0.411765,0.411765,0.176471,0.000000,dir-explicit,0.411765
3,가/JKS,112,0.0,0.375000,0.312500,0.285714,0.026786,dir-explicit,0.375000
4,가/VV,11,0.0,0.000000,0.818182,0.090909,0.090909,dir-speech,0.818182


In [5]:
dic_inten.head()

,ngram,freq,High,Low,Medium,None,max.value,max.prop
0,가*/JKS,1,0.000000,0.000000,1.000000,0.000000,Medium,1.000000
1,가*/VV,3,0.000000,0.333333,0.666667,0.000000,Medium,0.666667
2,가/JKC,17,0.176471,0.235294,0.588235,0.000000,Medium,0.588235
3,가/JKS,112,0.160714,0.107143,0.696429,0.035714,Medium,0.696429
4,가/VV,11,0.090909,0.181818,0.636364,0.090909,Medium,0.636364


In [6]:
dic_polar.head()

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
1,가*/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
2,가*/JKS;있/VV;었/EP,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3,가*/VV,3,0.0,0.0,0.0,0.0,1.0,POS,1.0
4,가*/VV;ㄴ다*/EF,1,0.0,0.0,0.0,0.0,1.0,POS,1.0


In [7]:
regex = re.compile("가.*")
regex.findall("촉촉한 것이 정말 좋다.")

[]

In [8]:
dic_.loc[dic_["ngram"].isin(["가*/JKS"])]

NameError: name 'dic_' is not defined

In [9]:
dic_[dic_.ngram.apply(lambda x: regex.match(x)).str.len() > 0 ]

NameError: name 'dic_' is not defined

In [10]:
dic_extype[dic_extype.ngram.str.startswith("촉촉")]

,ngram,freq,dir-action,dir-explicit,dir-speech,indirect,writing-device,max.value,max.prop
3030,촉촉/XR,1,1.0,0.0,0.0,0.0,0.0,dir-action,1.0
9478,촉촉/XR;하/XSA,1,1.0,0.0,0.0,0.0,0.0,dir-action,1.0
15725,촉촉/XR;하/XSA;아/EC,1,1.0,0.0,0.0,0.0,0.0,dir-action,1.0


In [73]:
dic_inten[dic_inten.ngram.str.startswith("촉촉")]

,ngram,freq,High,Low,Medium,None,max.value,max.prop
3030,촉촉/XR,1,1.0,0.0,0.0,0.0,High,1.0
9477,촉촉/XR;하/XSA,1,1.0,0.0,0.0,0.0,High,1.0
15725,촉촉/XR;하/XSA;아/EC,1,1.0,0.0,0.0,0.0,High,1.0


In [5]:
dic_polar[dic_polar.ngram.str.contains("촉촉")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
14701,촉촉/XR,1,1.0,0.0,0.0,0.0,0.0,COMP,1.0
14702,촉촉/XR;하/XSA,1,1.0,0.0,0.0,0.0,0.0,COMP,1.0
14703,촉촉/XR;하/XSA;아/EC,1,1.0,0.0,0.0,0.0,0.0,COMP,1.0


In [ ]:
dic_polar[dic_polar.ngram.str.contains("촉촉")]

In [76]:
dic_polar[dic_polar.ngram.str.startswith("않")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
9203,않/VV,1,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000
9204,않/VX,88,0.011364,0.556818,0.022727,0.056818,0.352273,NEG,0.556818
9205,않/VX;겠/EP,3,0.000000,0.666667,0.000000,0.000000,0.333333,NEG,0.666667
9206,않/VX;겠/EP;다고/EC,1,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000
9207,않/VX;겠/EP;지요/EF,1,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000
9208,않/VX;고/EC,4,0.000000,0.250000,0.250000,0.000000,0.500000,POS,0.500000
9209,않/VX;고/EC;최고/NNG,1,0.000000,1.000000,0.000000,0.000000,0.000000,NEG,1.000000
9210,않/VX;기/ETN,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
9211,않/VX;기/ETN;위하/VV,1,0.000000,0.000000,0.000000,0.000000,1.000000,POS,1.000000
9212,않/VX;는/ETM,3,0.000000,0.666667,0.000000,0.000000,0.333333,NEG,0.666667


In [80]:
dic_polar[dic_polar.ngram.str.startswith("민감")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop


In [6]:
dic_polar[dic_polar.ngram.str.contains("최악")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
14747,최악/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [81]:
dic_polar[dic_polar.ngram.str.startswith("하지")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
15694,하지만/MAJ,10,0.0,0.1,0.0,0.8,0.1,None,0.8
15695,하지만/MAJ;꼭/MAG,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
15696,하지만/MAJ;꼭/MAG;필요/NNG,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
15697,하지만/MAJ;남편/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
15698,하지만/MAJ;남편/NNG;은/JX,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [83]:
dic_polar[dic_polar.ngram.str.contains("민감")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop


In [84]:
dic_polar[dic_polar.ngram.str.contains("상품")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
7825,상품/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
7826,상품/NNG;미학/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
7827,상품/NNG;미학/NNG;이/VCP,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


### 형용사만 추출

In [89]:
dic_polar[dic_polar.ngram.str.contains("/XSN")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
27,가/JKS;극대/NNG;화/XSN,1,0.0,1.000000,0.0,0.0,0.000000,NEG,1.000000
186,가능*/NNG;성/XSN,1,0.0,1.000000,0.0,0.0,0.000000,NEG,1.000000
187,가능*/NNG;성/XSN;이/JKS,1,0.0,1.000000,0.0,0.0,0.000000,NEG,1.000000
189,가능/NNG;성/XSN,5,0.0,0.400000,0.6,0.0,0.000000,NEUT,0.600000
190,가능/NNG;성/XSN;의/JKG,1,0.0,1.000000,0.0,0.0,0.000000,NEG,1.000000
...,...,...,...,...,...,...,...,...,...
16237,효율/NNG;적/XSN;으로/JKB,3,0.0,0.333333,0.0,0.0,0.666667,POS,0.666667
16287,흑인/NNG;민/NNG;권/XSN,1,0.0,0.000000,0.0,0.0,1.000000,POS,1.000000
16300,흔히/MAG;들/XSN,1,0.0,0.000000,1.0,0.0,0.000000,NEUT,1.000000
16325,희극/NNG;적/XSN,2,0.0,0.500000,0.0,0.0,0.500000,NEG,0.500000


In [96]:
dic_polar[dic_polar.ngram.str.contains("/XR")]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
49,가/JKS;더욱/MAG;막중/XR,1,1.0,0.0,0.0,0.0,0.0,COMP,1.0
57,가/JKS;막대/XR,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
58,가/JKS;막대/XR;하/XSA,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
86,가/JKS;시급/XR,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
87,가/JKS;시급/XR;하/XSA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
...,...,...,...,...,...,...,...,...,...
16266,훨씬/MAG;한가/XR,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
16267,훨씬/MAG;한가/XR;하/XSA,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
16315,흥미진진/XR,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
16316,흥미진진/XR;하/XSA,1,0.0,0.0,0.0,0.0,1.0,POS,1.0


# Most사전

In [6]:
df_n = pd.read_excel(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_스킨 케어_Noun.xlsx")
df_adj = pd.read_excel(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_스킨 케어_Adj.xlsx")
df_ad = pd.read_excel(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_스킨 케어_Adverb.xlsx")
df_v = pd.read_excel(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_스킨 케어_Verb.xlsx")


In [9]:
df_ad.head()

,Unnamed: 0,word,count,setence_tag,setence,word2vec,Topic,TFIDF
0,0,더,4612,"[['ㅋㅋㅋ', '가볍', '톤', '업', '크림', '쿠션', '정도', '거뜬...","['뷰티 블 로 거 라벤더에요 :) 이미 시작된 여름, 자외선이 한층 강해 지면서 ...","[('훨씬', 0.6231709122657776), ('더욱', 0.61781728...","[('더', 230), ('하', 128), ('피부', 120), ('있', 98...",-439.570549
1,1,잘,2664,"[['주', '안', '쓰', '없', '포인트', '제품', '절', '아이', ...",['포인트 제품 중에 저는 아이 리무버를 정말 잘 썼는데클렌저는 처음? 일 거예요!...,"[('맞', 0.68433678150177), ('쪼개지', 0.6170310974...","[('잘', 178), ('피부', 136), ('하', 112), ('있', 11...",-253.906319
2,2,정말,1873,"[['비타민', 'C', '답', '노랑색', '패키지', '레몬', '라', '임...",['그런데! 망가진 제 피부에 피부 톤이라도 밝혀 주면서 노폐물까지 제거해 준다 길...,"[('진짜', 0.7102656960487366), ('너무', 0.63828957...","[('정말', 236), ('있', 124), ('피부', 116), ('하', 1...",-178.515967
3,3,없이,1789,"[['톤', '개선', '도움', '자극', '제품', '입증', '같이', '예민...",['포인트 딥 클린 비 타 휩 클렌징폼은 순도 99% 비타민 C( 아 스코 빅 애 ...,"[('스슥', 0.6481261849403381), ('없', 0.630111336...","[('피부', 160), ('있', 160), ('주', 100), ('사용', 1...",-170.509912
4,4,너무,1637,"[['바로', '사용하', '보', '클렌징', '매일', '하', '일', '톤'...","['클렌징은 매일 하는 일인데 톤 업까지 된다면 너무 좋잖아요.', '피부 톤 뿐만...","[('진짜', 0.6486830711364746), ('정말', 0.63828957...","[('하', 130), ('피부', 116), ('좋', 112), ('있', 11...",-156.022764


In [15]:
dic_polar[dic_polar.ngram.str.startswith("잘")].iloc[0,]

ngram            잘/MAG
freq                14
COMP                 0
NEG           0.214286
NEUT         0.0714286
None                 0
POS           0.714286
max.value          POS
max.prop      0.714286
Name: 13164, dtype: object

In [16]:
dic_polar[dic_polar.ngram.str.startswith("정말")].iloc[0,]

ngram        정말/MAG
freq              4
COMP              0
NEG               0
NEUT              0
None           0.25
POS            0.75
max.value       POS
max.prop       0.75
Name: 13638, dtype: object

In [17]:
df_v.head()

,Unnamed: 0,word,count,setence_tag,setence,word2vec,Topic,TFIDF
0,0,있,17923,"[['어', '충분히', '거품', '만들', '주', '다음', '만들', '지'...",['하는데요. 사용 방법은 내용물 2-3cm 정도를 손바닥에 덜어 내 어 충분히 거...,"[('되', 0.727159857749939), ('하', 0.70101892948...","[('있', 294), ('피부', 140), ('하', 94), ('제품', 80...",-1708.244353
1,1,하,15181,"[['맑', '환', '피부', '연출', '휘', '미세거품이', '노폐물', '...","[""[' 안녕하세요"", '뷰티 블 로 거 라벤더에요 :) 이미 시작된 여름, 자외선...","[('되', 0.7286673784255981), ('주', 0.7135457992...","[('하', 328), ('피부', 146), ('있', 112), ('보', 94...",-1446.903840
2,2,주,12863,"[['지', '피부', '점점', '칙칙', '해지', '느끼', '피부', '톤'...","['뷰티 블 로 거 라벤더에요 :) 이미 시작된 여름, 자외선이 한층 강해 지면서 ...","[('하', 0.7135457992553711), ('피부', 0.687795817...","[('주', 290), ('피부', 212), ('하', 156), ('있', 11...",-1225.974843
3,3,좋,7326,"[['거품', '일어나', '굳이', '거품', '망', '필요', '없', '고요...",['★ 사용감을 보여드리기 위해 평소에 사용하는 파운데이션을 손등에 펴 발라 준 다...,"[('같', 0.7001716494560242), ('낭창', 0.676133751...","[('좋', 304), ('피부', 142), ('하', 140), ('있', 11...",-698.242377
4,4,보,7310,"[['정도', '거뜬히', '지우', '좀', '더', '자세히', '보여드리', ...",['★ 사용감을 보여드리기 위해 평소에 사용하는 파운데이션을 손등에 펴 발라 준 다...,"[('보시', 0.6097292900085449), ('로지샤틴크림', 0.5965...","[('보', 280), ('피부', 118), ('하', 106), ('있', 92...",-696.717414


In [23]:
dic_polar[dic_polar.ngram.str.startswith("보")].iloc[0,]

ngram        보*/VV
freq             2
COMP             0
NEG            0.5
NEUT             0
None             0
POS            0.5
max.value      NEG
max.prop       0.5
Name: 7024, dtype: object

In [28]:
dic_polar[dic_polar.ngram.str.startswith("보")].iloc[0,]["max.value"]

'NEG'

In [34]:
def combine_voca(version):
    #version = "v"

    df_ = globals() ["df_{}".format(version)]

    #globals() ["polarity_{}".format(version)]  = []
    #globals() ["probs_{}".format(version)]  = []

    polarity = []
    probs = []

    for i in df_["word"]:
        try:
            pol = dic_polar[dic_polar.ngram.str.startswith(i)].iloc[0,]["max.value"]
            prob = dic_polar[dic_polar.ngram.str.startswith(i)].iloc[0,]["max.prop"]
            polarity.append(pol)
            probs.append(prob)
            #print(pol)
        except:
            polarity.append("None")
            probs.append("None")
    df_2 = df_.loc[:,["word","count","TFIDF"]]

    df_2["polarity"] = polarity
    df_2["probs"] = probs
    globals() ["total_{}".format(version)] = df_2

In [35]:
v_list = ["v","adj","ad","n"]
for v in v_list:
    combine_voca(v)

# combine 중에서 어떤 데이터를 사용해야 하나?

### 1. ad

In [42]:
display(total_ad.head())
total_ad["polarity"].value_counts()

,word,count,TFIDF,polarity,probs
0,더,4612,-439.570549,POS,0.695652
1,잘,2664,-253.906319,POS,0.714286
2,정말,1873,-178.515967,POS,0.75
3,없이,1789,-170.509912,NEG,0.470588
4,너무,1637,-156.022764,NEG,0.714286


None    264
NEG      63
POS      60
NEUT      8
COMP      5
Name: polarity, dtype: int64

In [47]:
total_ad[total_ad["polarity"]=="POS"]

,word,count,TFIDF,polarity,probs
0,더,4612,-439.570549,POS,0.695652
1,잘,2664,-253.906319,POS,0.714286
2,정말,1873,-178.515967,POS,0.75
6,다,1470,-140.105964,POS,1
11,함께,1167,-111.226980,POS,0.6
12,좀,1125,-107.223952,POS,1
16,꼭,716,-68.242089,POS,0.666667
17,가장,715,-68.146779,POS,0.612903
18,아주,685,-65.287473,POS,0.75
29,먼저,547,-52.134668,POS,0.75


In [48]:
total_ad[total_ad["polarity"]=="NEG" and total_ad["polarity"]=="NEG"]

,word,count,TFIDF,polarity,probs
3,없이,1789,-170.509912,NEG,0.470588
4,너무,1637,-156.022764,NEG,0.714286
5,많이,1637,-156.022764,NEG,0.555556
9,안,1240,-118.184623,NEG,0.5625
10,그리고,1183,-112.751943,NEG,0.5
...,...,...,...,...,...
356,제발,17,8.684036,NEG,1
358,그저,16,3.570297,NEG,0.5
359,척,16,5.706799,NEG,1
372,흐,15,0.000000,NEG,0.8


In [43]:
display(total_adj.head())
total_adj["polarity"].value_counts()

,word,count,TFIDF,polarity,probs
0,촉촉,3342,-318.526621,COMP,1
1,산뜻,985,0.000000,POS,1
2,민감,773,0.000000,None,None
3,예민,753,0.000000,None,None
4,매끈,592,0.000000,POS,1


None    290
POS      49
NEG      28
NEUT      4
COMP      3
Name: polarity, dtype: int64

In [45]:
display(total_v.head())
total_v["polarity"].value_counts()

,word,count,TFIDF,polarity,probs
0,있,17923,-1708.244353,POS,0.75
1,하,15181,-1446.903840,POS,0.5
2,주,12863,-1225.974843,POS,0.6
3,좋,7326,-698.242377,POS,0.92
4,보,7310,-696.717414,NEG,0.5


None    147
NEG     121
POS     117
NEUT     10
COMP      5
Name: polarity, dtype: int64

In [46]:
display(total_n.head())
total_n["polarity"].value_counts()

,word,count,TFIDF,polarity,probs
0,피부,21815,-2079.191572,POS,1
1,사용,8818,-840.445166,NEG,0.333333
2,제품,8223,-783.735609,NEUT,0.666667
3,크림,7263,1620.691613,POS,1
4,수분,6214,0.000000,NEG,1


None    200
POS     103
NEG      79
NEUT     11
COMP      7
Name: polarity, dtype: int64